# Clustering Using Pairwise Comparison

Clustering GP bitmaps

## Application to Preference Shopping

According to the work of R. Srikant, it is possible to cluster users based on incomplete information on their purchasing preferences. They achieve this by converting an (Mx2) matrix about the choices of users over products into an (M) 'net wins matrix'.

For instance assume we have 4 products (r1, r2, r3, r4) and 3 users (u1, u2, u3). We use the following encoding: if a user picks r1 over r2 (r1 > r2) then (r1,r2)=1 and (r2,r1)=-1; if we do not have a preferred choice (this accounts for incomplete information) between r3 and r4 then the encoding is (r3,r4)=0.

A sample matrix may take the form:

| u |(r1,r2) | (r1,r3) | (r1,r4) | (r2,r3) | (r2,r4) | (r3,r4) |
| :--- | :---: | :---: | :---: | :---: | :---: | :---: |
| u1 | 1 | 0 | 0 | -1 | 1 | 0 |
| u2 | 0 | 1 | 0 | -1 | 0 | 0 |
| u3 | 1 | -1 | 0 | 0 | 1 | 0 |


Using the available preference information, we can convert this matric into a 'net wins matrix' as shown below (i.e., for each item if it has more wins than losses then it is 1; if it has more losses than wins then it is -1; if the losses and equal to wins then it is 0; if it has no information then it is 0)

| u | r1 | r2 | r3 | r4 |
| --- | --- | --- | --- | --- |
| u1 | 1 | -1 | 1 | -1 |
| u2 | 1 | -1 | 0 | 0 |
| u3 | 0 | 0 | 1 | -1 |

We can use this new matrix to cluster users based on their purchase preferences using spectral clustering.

## Application to GP Mining

In order to mine for GPs, each feature has to construct a bitmap matrix comparing each row to each other (i.e., (r1,r2), (r1,r3), (r1,r4), (r2,r3), (r2,r4), (r3,r4)).

We can apply the same approach here, such that, we only construct the bitmap matrix with a handful of rows then convert them into 'net wins matrices'. Finally, we apply spectral clustering to determine which features belong to the same group based on the similarity of gradual dependency.

# Install Libraries

In [1]:
# Installing Libraries (if not installed)
#!pip3 install jupyterlab-git
#!pip3 install voila
#!pip3 install matplotlib
#!pip3 install scikit-learn
#!pip3 install seaborn
#!pip3 install numpy
#!pip3 install pandas
#!pip3 install so4gp
#!pip3 install scipy
#!pip3 install sympy
#!pip3 install networkx

# Import libraries <a name="lib-import"></a>

In [54]:
# Importing Libraries
#import pandas as pd
import numpy as np
#import scipy
#import math

from numpy.linalg import svd


In [55]:
import so4gp as sgp

# Data set
Retrieving data set and the features

In [56]:
ds = sgp.DataGP('../data/DATASET.csv')
#ds = sgp.DataGP('../data/c2k_02k.csv')
#ds = sgp.DataGP('../data/breast_cancer.csv')


data_size = ds.row_count
feature_size = ds.col_count
features = np.array(ds.data, dtype=np.float64)
#features

In [57]:
# Mine GPs using existing GRAANK algorithm
#sgp.graank('../data/breast_cancer.csv', min_sup=0.6)

# Clustering GPs using Gradual Pairs

## a. Constructing Pair Matrix
We construct gradual pairs using the following encoding:

if r1 > r2, then (r1,r2) = 1

if r1 < r2 then (r1,r2) = -1

if r1 == r2 then (r1,r2) = 0


In [58]:
# Function for constructing GP pairs for Mx2 matrix

def construct_pairs():
    X = []
    for i in range(data_size):
        for j in range(i+1, data_size):
            x = []
            for k in range(feature_size):
                if features[j][k] > features[i][k]:
                    encoding = 1
                elif features[j][k] < features[i][k]:
                    encoding = -1
                else:
                    encoding = 0
                x.append(encoding)
            X.append(x)
    return np.array(X, dtype=float).T

def construct_bm_pairs():
    attr_data = features.T
    bm_data = []
    for c_data in attr_data:
        col_data = np.array(c_data, dtype=float)
        pair_bitmap = np.where(col_data > col_data[:, np.newaxis], 1, np.where(col_data < col_data[:, np.newaxis], -1, 0))
        bm_data.append(pair_bitmap)
    return bm_data

In [59]:
M = construct_pairs()
print(M)

Mp = construct_bm_pairs()
Mp

[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [-1.  1. -1.  1.  1. -1.  1. -1.  1.  1.]
 [ 1.  1.  0.  0.  0. -1. -1. -1. -1.  0.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


[array([[ 0,  1,  1,  1,  1],
        [-1,  0,  1,  1,  1],
        [-1, -1,  0,  1,  1],
        [-1, -1, -1,  0,  1],
        [-1, -1, -1, -1,  0]]),
 array([[ 0, -1,  1, -1,  1],
        [ 1,  0,  1, -1,  1],
        [-1, -1,  0, -1,  1],
        [ 1,  1,  1,  0,  1],
        [-1, -1, -1, -1,  0]]),
 array([[ 0,  1,  1,  0,  0],
        [-1,  0,  0, -1, -1],
        [-1,  0,  0, -1, -1],
        [ 0,  1,  1,  0,  0],
        [ 0,  1,  1,  0,  0]]),
 array([[ 0, -1, -1, -1, -1],
        [ 1,  0, -1, -1, -1],
        [ 1,  1,  0, -1, -1],
        [ 1,  1,  1,  0, -1],
        [ 1,  1,  1,  1,  0]]),
 array([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]),
 array([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]])]

## b. Bulding the Net Wins Matrix

We construct a Net Wins matrix as follows

In [60]:
def construct_net_wins(M):
    N = []
    for bm in M:
        R = []
        for r in bm:
            s = np.sum(r)
            if s > 0:
                s = 1
            elif s < 0:
                s = -1
            R.append(s)
        N.append(R)
    return np.array(N)

In [62]:
N = construct_net_wins(Mp)
N

array([[ 1,  1,  0, -1, -1],
       [ 0,  1, -1,  1, -1],
       [ 1, -1, -1,  1,  1],
       [-1, -1,  0,  1,  1],
       [ 0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0]])

## c. Compute Singular Value Decomposition values

In [63]:
U, S, VT = svd(N)
S

array([3.15462523e+00, 2.06739929e+00, 1.66559292e+00, 5.76694498e-17,
       0.00000000e+00])